In [1]:
import numpy as np
import plotly.graph_objects as go
from enum import Enum

In [2]:
class Comoditie(Enum):
    MINIMAL_NON_REACHED = (-2, "Sem solução: região factível além da fronteira.")
    MINIMAL_INVIABLE = (-1, "Sem solução. Minimo inatingivel.")
    NOTHING = (0, "Sem solução")
    SUGAR = (1, "Açucar.")
    ETANOL = (2, "Etanol.")
    BOTH = (3, "Açucar e etanol.")

    def __new__(cls, value, description):
        obj = object.__new__(cls)
        obj._value_ = value
        obj.description = description
        return obj

    @property
    def desc(self):
        return self.description

In [64]:
def geometric_solution(
    s, k, c1, c2, a11, a12,
    comodities_intervals=200,
    profits_lines_qnt=4,
    sugar_min_contract=0,
    etanol_min_contract=0
):
    best_option = Comoditie.NOTHING

    # maximo produzivel de cana
    t = s * k  

    #  maximo produziveis por comodite, sem minimo especificado
    sugar_max = t / a11
    etanol_max = t / a12

    # Criando grafico
    fig = go.Figure()
    fig.update_layout(
        title="Região factível com contratos e linhas de nível",
        xaxis_title="ton açúcar",
        yaxis_title="1000L etanol",
        template='plotly_white'
    )

    # checando inviabilidade das quantidades mínimas obrigatórias 
    if sugar_min_contract * a11 + etanol_min_contract * a12 > t:
        fig.update_layout(
            annotations=[
                dict(
                    x=0.5, y=1.10, xref="paper", yref="paper",
                    text=Comoditie.MINIMAL_INVIABLE.desc,
                    showarrow=False,
                    font=dict(size=14)
                )
            ]
        )   
        best_option = Comoditie.MINIMAL_INVIABLE

    # intervalos de producoes das comodities distribuidas num range de 0 ao maximo daquela comoditie, em passos de 'comodities_distribuion'
    sugar_intervals_productions = np.linspace(0, sugar_max, comodities_intervals)
    etanol_intervals_productions = (t - (a11 * sugar_intervals_productions)) / a12
    etanol_intervals_productions_complement = np.clip(etanol_intervals_productions, 0, None)

    # demarcando a fronteira
    fig.add_trace(go.Scatter(
        x=sugar_intervals_productions,
        y=etanol_intervals_productions_complement,
        name="Fronteira"
    ))
    # demarcando a regiao factivel com minimo aplicado
    fig.add_trace(go.Scatter(
        x=[sugar_min_contract, sugar_max, sugar_min_contract, sugar_min_contract],
        y=[etanol_min_contract, etanol_min_contract, etanol_max, etanol_min_contract],
        fill='toself',
        fillcolor='rgba(0,100,200,0.15)',
        line=dict(color='rgba(0,0,0,0.3)'),
        name='Região factível minima'
    ))

    # calculando producao com aplicacao de quantidade minima
    sugar_feasible = sugar_intervals_productions[sugar_intervals_productions >= sugar_min_contract]
    etanol_feasible = etanol_intervals_productions_complement[sugar_intervals_productions >= sugar_min_contract]
    etanol_feasible = np.where(etanol_feasible >= etanol_min_contract, etanol_feasible, np.nan)

    # obtendo dados sobre o resultado
    sugar_range = sugar_feasible
    etanol_range = etanol_feasible
    best_profit = c1 * sugar_range + c2 * etanol_range
    mask = ~np.isnan(best_profit)

    # verificando encontro da regiao factivel com valores minimos
    if mask.sum() == 0:
        fig.update_layout(
            annotations=[
                dict(
                    x=0.5, y=1.10, xref="paper", yref="paper",
                    text=Comoditie.MINIMAL_NON_REACHED.desc,
                    showarrow=False,
                    font=dict(size=14)
                )
            ]
        )   
        best_option = Comoditie.MINIMAL_NON_REACHED

    # Obtendo melhores valores
    if best_option.value >= 0:
        idx = np.argmax(best_profit[mask])
        best_sugar = sugar_range[mask][idx]
        best_etanol = etanol_range[mask][idx]
        best_profit = best_profit[mask][idx]
        best_option = Comoditie.BOTH
        if best_sugar == 0: 
            best_option = Comoditie.ETANOL
        elif best_etanol == 0:
            best_option = Comoditie.SUGAR

        # adicionando marcacao de minimo atingido
        fig.update_layout(
            annotations=[
                dict(
                    x=0.5, y=1.10, xref="paper", yref="paper",
                    text=f"Minimo atingido. Lucro={best_profit:.2f}",
                    showarrow=False,
                    font=dict(size=14)
                ),
                dict(
                    x=0.5, y=1.05, xref="paper", yref="paper",
                    text=f"Deve produzir: {best_option.desc}",
                    showarrow=False,
                    font=dict(size=14)
                )
            ]
        )

        # marcando solucao otima no grafico
        if best_option == Comoditie.BOTH:
            fig.add_trace(go.Scatter(
                x=sugar_intervals_productions,
                y=etanol_intervals_productions_complement,
                mode="lines",
                marker=dict(size=12, color="red"),
                name=f"Ótimo"
            ))
            better_option_text = f"açúcar={best_sugar:.2f}T, etanol={best_etanol:.2f}L"
        elif best_option == Comoditie.SUGAR or best_option == Comoditie.ETANOL:
            fig.add_trace(go.Scatter(
                x=[best_sugar],
                y=[best_etanol],
                mode="markers",
                marker=dict(size=12, color="red"),
                name=f"Ótimo"
            ))

    # adicionando 'profits_lines_qnt' linhas de nivel ao grafico
    if profits_lines_qnt > 0:
        sugar_profit_max = c1 * sugar_max
        etanol_profit_max = c2 * etanol_max
        profit_intervals = np.linspace(0, max(sugar_profit_max, etanol_profit_max), profits_lines_qnt+1)
        profit_intervals = profit_intervals[1:]
        if best_option != Comoditie.SUGAR and best_option != Comoditie.ETANOL:
            profit_intervals = profit_intervals[:-1]
        
        profit_sugar_etanol_intervals = []
        for intervals in profit_intervals:
            sugar_interval_profit_line = np.linspace(0, sugar_max, comodities_intervals)
            etanol_interval_profit_line = (intervals - c1 * sugar_interval_profit_line) / c2
            etanol_interval_profit_line_visible = np.where(etanol_interval_profit_line >= 0, etanol_interval_profit_line, np.nan)
            profit_sugar_etanol_intervals.append((sugar_interval_profit_line, etanol_interval_profit_line_visible))
        for i, (sugar, etanol) in enumerate(profit_sugar_etanol_intervals):
            fig.add_trace(go.Scatter(
                x=sugar,
                y=etanol,
                mode='lines',
                line=dict(dash='dot'),
                name=f"linha {i+1}"
            ))

    return fig, best_option


In [65]:
# Cenario de acucar

c1_sugar = 220      
c2_sugar = 50         
a11_sugar = 8.0         
a12_sugar = 2.0         
s_sugar = 100.0         
k_sugar = 80.0

(fig, most_profitable)  = geometric_solution(s_sugar, k_sugar, c1_sugar, c2_sugar, a11_sugar, a12_sugar)
fig

In [66]:
# Cenario de etanol

c1_etanol = 45
c2_etanol = 230         
a11_etanol = 7.0         
a12_etanol = 3.0         
s_etanol = 100.0         
k_etanol = 80.0

(fig, most_profitable)  = geometric_solution(s_etanol, k_etanol, c1_etanol, c2_etanol, a11_etanol, a12_etanol)
fig

In [67]:
# Cenario de igualdade

c1_equal = 200      
c2_equal = 200         
a11_equal = 4.0         
a12_equal = 4.0         
s_equal = 100.0         
k_equal = 80.0

(fig, most_profitable)  = geometric_solution(s_equal, k_equal, c1_equal, c2_equal, a11_equal, a12_equal)
fig

In [68]:
# Cenario de minimo inatingivel

c1_intagible = 200      
c2_intagible = 200         
a11_intagible = 4.0         
a12_intagible = 4.0         
s_intagible = 100.0         
k_intagible = 80.0
m1 = 3000
m2 = 3000

(fig, most_profitable)  = geometric_solution(s_intagible, k_intagible, c1_intagible, c2_intagible, a11_intagible, a12_intagible, sugar_min_contract=m1, etanol_min_contract=m2)
fig

In [69]:
# Cenario de minimo nao alcancado

c1_non_reach = 200      
c2_non_reach = 200         
a11_non_reach = 4.0         
a12_non_reach = 4.0         
s_non_reach = 100.0         
k_non_reach = 80.0
m1 = 1000
m2 = 1000

(fig, most_profitable)  = geometric_solution(s_non_reach, k_non_reach, c1_non_reach, c2_non_reach, a11_non_reach, a12_non_reach, sugar_min_contract=m1, etanol_min_contract=m2)
fig